# The capital asset pricing model and the security market line

In [7]:
""" 
Linear regression with SciPy 
"""
from scipy import stats

stock_returns = [0.065, 0.0265, -0.0593, -0.001, 0.0346]
mkt_returns = [0.055, -0.09, -0.041, 0.045, 0.022]
beta, alpha, r_value, p_value, std_err = \
    stats.linregress(stock_returns, mkt_returns)

In [8]:
print(beta, alpha)

0.5077431878770808 -0.008481900352462384


# Multivariate linear regression of factor models

In [9]:
""" 
Least squares regression with statsmodels 
"""
import numpy as np
import statsmodels.api as sm

# Generate some sample data
num_periods = 9
all_values = np.array([np.random.random(8) \
                       for i in range(num_periods)])

# Filter the data
y_values = all_values[:, 0] # First column values as Y
x_values = all_values[:, 1:] # All other values as X
x_values = sm.add_constant(x_values) # Include the intercept
results = sm.OLS(y_values, x_values).fit() # Regress and fit the model

In [10]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                     1117.
Date:                Sun, 19 Aug 2018   Prob (F-statistic):             0.0230
Time:                        00:30:24   Log-Likelihood:                 38.819
No. Observations:                   9   AIC:                            -61.64
Df Residuals:                       1   BIC:                            -60.06
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1570      0.021      7.335      0.0

c:\python37\lib\site-packages\scipy\stats\stats.py:1394: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  "anyway, n=%i" % int(n))


In [11]:
print(results.params)

[ 0.15700937 -0.45126648  0.84676324  0.13568198 -0.57281289  0.98829748
  0.14180069 -0.44376925]


# Linear optimization

## A maximization example with linear programming

In [15]:
""" 
A linear optimization problem with 2 variables
"""
import pulp

x = pulp.LpVariable('x', lowBound=0)
y = pulp.LpVariable('y', lowBound=0)

problem = pulp.LpProblem(
    'A simple maximization objective', 
    pulp.LpMaximize)
problem += 3*x + 2*y, 'The objective function'
problem += 2*x + y <= 100, '1st constraint'
problem += x + y <= 80, '2nd constraint'
problem += x <= 40, '3rd constraint'
problem.solve()

1

In [16]:
print("Maximization Results:")
for variable in problem.variables():
    print(variable.name, '=', variable.varValue)

Maximization Results:
x = 20.0
y = 60.0


## A minimization example with integer programming

In [1]:
""" 
An example of implementing an integer 
programming model with binary conditions 
"""
import pulp

dealers = ['X', 'Y', 'Z']
variable_costs = {'X': 500, 'Y': 350, 'Z': 450}
fixed_costs = {'X': 4000, 'Y': 2000, 'Z': 6000}

# Define PuLP variables to solve
quantities = pulp.LpVariable.dicts('quantity', 
                                   dealers, 
                                   lowBound=0,
                                   cat=pulp.LpInteger)
is_orders = pulp.LpVariable.dicts('orders', 
                                  dealers,
                                  cat=pulp.LpBinary)

In [3]:
"""
This is an example of implementing an integer programming model
with binary variables the wrong way.
"""
# Initialize the model with constraints
model = pulp.LpProblem('A cost minimization problem',
                       pulp.LpMinimize)
model += sum([(variable_costs[i] * \
               quantities[i] + \
               fixed_costs[i])*is_orders[i] \
              for i in dealers]), 'Minimize portfolio cost'
model += sum([quantities[i] for i in dealers]) == 150\
    , 'Total contracts required'
model += 30 <= quantities['X'] <= 100\
    , 'Boundary of total volume of X'
model += 30 <= quantities['Y'] <= 90\
    , 'Boundary of total volume of Y'
model += 30 <= quantities['Z'] <= 70\
    , 'Boundary of total volume of Z'
model.solve()  # You will get an error running this code!

TypeError: Non-constant expressions cannot be multiplied

In [19]:
"""
This is an example of implementing an 
IP model with binary variables the correct way.
"""
# Initialize the model with constraints
model = pulp.LpProblem('A cost minimization problem',
                       pulp.LpMinimize)
model += sum(
    [variable_costs[i]*quantities[i] + \
         fixed_costs[i]*is_orders[i] for i in dealers])\
    , 'Minimize portfolio cost'
model += sum([quantities[i] for i in dealers]) == 150\
    ,  'Total contracts required'
model += is_orders['X']*30 <= quantities['X'] <= \
    is_orders['X']*100, 'Boundary of total volume of X'
model += is_orders['Y']*30 <= quantities['Y'] <= \
    is_orders['Y']*90, 'Boundary of total volume of Y'
model += is_orders['Z']*30 <= quantities['Z'] <= \
    is_orders['Z']*70, 'Boundary of total volume of Z'
model.solve()

1

In [20]:
print('Minimization Results:')
for variable in model.variables():
    print(variable, '=', variable.varValue)

print('Total cost:',  pulp.value(model.objective))

Minimization Results:
orders_X = 0.0
orders_Y = 1.0
orders_Z = 1.0
quantity_X = 0.0
quantity_Y = 90.0
quantity_Z = 60.0
Total cost: 66500.0


# Solving linear equations using matrices

In [43]:
""" 
Linear algebra with NumPy matrices 
"""
import numpy as np

A = np.array([[2, 1, 1],[1, 3, 2],[1, 0, 0]])
B = np.array([4, 5, 6])

Use the `linalg.solve` function of NumPy to solve a system of linear scalar
equations:

In [44]:
print(np.linalg.solve(A, B))

[  6.  15. -23.]


# The LU decomposition

In [45]:
 """ 
LU decomposition with SciPy 
"""
import numpy as np
import scipy.linalg as linalg


# Define A and B
A = np.array([
    [2., 1., 1.],
    [1., 3., 2.],
    [1., 0., 0.]])
B = np.array([4., 5., 6.])

# Perform LU decompositiopn
LU = linalg.lu_factor(A)
x = linalg.lu_solve(LU, B)

In [26]:
print(x)

[  6.  15. -23.]


In [27]:
import scipy

P, L, U = scipy.linalg.lu(A)

print('P=\n', P)
print('L=\n', L)
print('U=\n', U)

P=
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
L=
 [[ 1.   0.   0. ]
 [ 0.5  1.   0. ]
 [ 0.5 -0.2  1. ]]
U=
 [[ 2.   1.   1. ]
 [ 0.   2.5  1.5]
 [ 0.   0.  -0.2]]


# The Cholesky decomposition

In [28]:
""" 
Cholesky decomposition with NumPy 
"""
import numpy as np

A = np.array([
    [10., -1., 2., 0.],
    [-1., 11., -1., 3.],
    [2., -1., 10., -1.],
    [0., 3., -1., 8.]])
B = np.array([6., 25., -11., 15.])

L = np.linalg.cholesky(A)

In [29]:
print(L)

[[ 3.16227766  0.          0.          0.        ]
 [-0.31622777  3.3015148   0.          0.        ]
 [ 0.63245553 -0.24231301  3.08889696  0.        ]
 [ 0.          0.9086738  -0.25245792  2.6665665 ]]


In [30]:
print(np.dot(L, L.T.conj()))  # A=L.L*

[[10. -1.  2.  0.]
 [-1. 11. -1.  3.]
 [ 2. -1. 10. -1.]
 [ 0.  3. -1.  8.]]


In [31]:
y = np.linalg.solve(L, B)  # L.L*.x=B; When L*.x=y, then L.y=B

In [32]:
x = np.linalg.solve(L.T.conj(), y)  # x=L*'.y

In [33]:
print(x)

[ 1.  2. -1.  1.]


In [34]:
print(np.mat(A) * np.mat(x).T)  # B=Ax

[[  6.]
 [ 25.]
 [-11.]
 [ 15.]]


# The QR decomposition

In [35]:
""" 
QR decomposition with scipy 
"""
import numpy as np
import scipy.linalg as linalg


A = np.array([
    [2., 1., 1.],
    [1., 3., 2.],
    [1., 0., 0]])
B = np.array([4., 5., 6.])

Q, R = scipy.linalg.qr(A)  # QR decomposition
y = np.dot(Q.T, B)  # Let y=Q'.B
x = scipy.linalg.solve(R, y)  # Solve Rx=y

In [36]:
print(x)

[  6.  15. -23.]


# Solving with other matrix algebra methods

## The Jacobi method

In [37]:
"""
Solve Ax=B with the Jacobi method 
"""
import numpy as np

def jacobi(A, B, n, tol=1e-10):
    # Initializes x with zeroes with same shape and type as B
    x = np.zeros_like(B)

    for iter_count in range(n):
        x_new = np.zeros_like(x)
        for i in range(A.shape[0]):
            s1 = np.dot(A[i, :i], x[:i])
            s2 = np.dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (B[i] - s1 - s2) / A[i, i]

        if np.allclose(x, x_new, tol):
            break

        x = x_new

    return x

In [38]:
A = np.array([
    [10., -1., 2., 0.], 
    [-1., 11., -1., 3.], 
    [2., -1., 10., -1.], 
    [0.0, 3., -1., 8.]])
B = np.array([6., 25., -11., 15.])
n = 25

In [39]:
x = jacobi(A, B, n)
print('x', '=', x)

x = [ 1.  2. -1.  1.]


# The Gauss-Seidel method

In [40]:
""" 
Solve Ax=B with the Gauss-Seidel method 
"""
import numpy as np


def gauss(A, B, n, tol=1e-10):
    L = np.tril(A)  # returns the lower triangular matrix of A
    U = A-L  # decompose A = L + U
    L_inv = np.linalg.inv(L)
    x = np.zeros_like(B)

    for i in range(n):
        Ux = np.dot(U, x)
        x_new = np.dot(L_inv, B - Ux)

        if np.allclose(x, x_new, tol):
            break

        x = x_new

    return x

In [41]:
A = np.array([
    [10., -1., 2., 0.], 
    [-1., 11., -1., 3.], 
    [2., -1., 10., -1.], 
    [0.0, 3., -1., 8.]])
B = np.array([6., 25., -11., 15.])
n = 100
x = gauss(A, B, n)

In [42]:
print('x', '=', x)

x = [ 1.  2. -1.  1.]
